There are 4 types of energy to predict <br>
    - 0 : electricity
    - 1 : chilledwater
    - 2 : steam
    - 3 : hotwater

Electricity and water consumption may have different behavior!<br>
     - I will make separately train & predict the model

* Reference 
 - https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type
 - https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction



In [1]:
# No leak data
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
%%time
# 파일 읽어오기
train_df = pd.read_csv('train.csv')
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_train_df = pd.read_csv('weather_train.csv')


test_df = pd.read_csv('test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

weather_test_df = pd.read_csv('weather_test.csv')
building_meta_df = pd.read_csv('building_metadata.csv')
sample_submission = pd.read_csv('sample_submission.csv')

Wall time: 58.8 s


In [3]:
# Glimpse of Data
print(train_df.shape)
print(weather_train_df.shape)
print(weather_test_df.shape)
print(building_meta_df.shape)

print(test_df.shape)

(20216100, 4)
(139773, 9)
(277243, 9)
(1449, 6)
(41697600, 4)


In [4]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
# Reducing memory
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)ㅏㅏ

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [6]:
def plot_date_usage(train_df, meter=0, building_id=0):
    train_temp_df = train_df[train_df['meter'] == meter]
    train_temp_df = train_temp_df[train_temp_df['building_id'] == building_id]    
    train_temp_df_meter = train_temp_df.groupby('date')['meter_reading_log1p'].sum()
    train_temp_df_meter = train_temp_df_meter.to_frame().reset_index()
    fig = px.line(train_temp_df_meter, x='date', y='meter_reading_log1p')
    fig.show()

## Feature engineering 
  - There are 3 parts to make features <br>
      train_df / weather_train_df / building_meta_df
  - and then I will merge them 

In [7]:
# train_df -- timestamp : 월 , 주, 일

train_df['meter_reading_log1p'] = np.log1p(train_df['meter_reading'])
# train_df['meter_reading']
# date / 월 / 주 /일 
train_df['date'] = train_df['timestamp'].dt.date
train_df['hour'] = train_df['timestamp'].dt.hour
train_df['weekend'] = train_df['timestamp'].dt.weekday
train_df['month'] = train_df['timestamp'].dt.month
train_df['dayofweek'] = train_df['timestamp'].dt.dayofweek


test_df['date'] = test_df['timestamp'].dt.date
test_df['hour'] = test_df['timestamp'].dt.hour
test_df['weekend'] = test_df['timestamp'].dt.weekday
test_df['month'] = test_df['timestamp'].dt.month
test_df['dayofweek'] = test_df['timestamp'].dt.dayofweek



In [8]:

weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [9]:
# weather - 
# weather data has a lot of nulls 
# I tried to fill these values by interpolating data
# df.groupby('').apply(lambda group: group.interpolate~~)

weather_train_df.head()
weather_train_df = weather_train_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))
weather_test_df = weather_test_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))

In [10]:

weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        17228
dew_temperature           0
precip_depth_1_hr     26273
sea_level_pressure     8755
wind_direction            0
wind_speed                0
dtype: int64

In [11]:
# lags 
# site 별로 최근 3일간의 날씨를 rolling 하기
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [12]:
add_lag_feature(weather_train_df, window=3)
add_lag_feature(weather_train_df, window=72)
add_lag_feature(weather_test_df, window=3)
add_lag_feature(weather_test_df, window=72)

In [13]:
# # meter reading 값에 대한 aggregation
# 과적합 문제를 야기할 수 있다.
# df_group = train_df.groupby('building_id')['meter_reading_log1p']
# building_mean = df_group.mean().astype(np.float16)
# building_median = df_group.median().astype(np.float16)
# building_min = df_group.min().astype(np.float16)
# building_max = df_group.max().astype(np.float16)
# building_std = df_group.std().astype(np.float16)

# train_df['building_mean'] = train_df['building_id'].map(building_mean)
# train_df['building_median'] = train_df['building_id'].map(building_median)
# train_df['building_min'] = train_df['building_id'].map(building_min)
# train_df['building_max'] = train_df['building_id'].map(building_max)
# train_df['building_std'] = train_df['building_id'].map(building_std)

In [14]:
# Removing weired data on site_id =0  
#https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113054#656588
# building_meta_df[building_meta_df.site_id == 0]

### Merge


In [15]:
# 하나에 합치기

# base + building_meta_df 합치기
train_df = pd.merge(train_df,building_meta_df, on= ['building_id'],how='left')
test_df = pd.merge(test_df,building_meta_df, on= ['building_id'],how='left')
# del building_meta_df

In [16]:
# base + weather_train_df 합치기
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

train_df = pd.merge(train_df,weather_train_df, on= ['site_id','timestamp'],how='left')
test_df = pd.merge(test_df,weather_test_df, on= ['site_id','timestamp'],how='left')
# del weather_train_df, weather_test_df

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import train_test_split

### Encoding



In [18]:
#Encoidng variables
le = LabelEncoder()
# train_df['primary_use'] = train_df['primary_use'].astype(str)
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

# test_df['primary_use'] = test_df['primary_use'].astype(str)
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype(np.int8)

In [19]:
# Pickle 저장

train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')
del train_df, test_df
gc.collect()



14

In [20]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [21]:
# some feature enginnering

train_df['age'] = train_df['year_built'].max()-train_df['year_built']+1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

In [22]:
#Handling missing values
# To streamline this though process it is useful to know the 3 categories in which missing data can be classified into:

# Missing Completely at Random (MCAR)
# Missing at Random (MAR)
# Missing Not at Random (MNAR)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 


In [23]:
drop_cols = ['date',"precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed","timestamp"]
target = train_df["meter_reading_log1p"]
del train_df["meter_reading"], train_df['meter_reading_log1p']
train_df = train_df.drop(drop_cols, axis=1)
drop_cols += ["row_id"]
# drop_cols.remove('date')
test_df = test_df.drop(drop_cols, axis=1)

In [24]:
import lightgbm as lgb
from sklearn.model_selection import KFold
# lightbgm
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'subsample': 0.2,
    'learning_rate': 0.08,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'alpha': 0.1,
    'lambda': 0.1,
#     'n_jobs' :2 
}


In [ ]:
%%time
folds = 7
seed = 99 #666
# shuffle = False
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)

models = []
for train_index, val_index in kf.split(train_df):
    train_X = train_df.iloc[train_index]
    val_X = train_df.iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]
    lgb_train = lgb.Dataset(train_X, train_y)
    lgb_eval = lgb.Dataset(val_X, val_y)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000, #300,
                    valid_sets=(lgb_train, lgb_eval),
#                     feval=rmsle,
                    early_stopping_rounds= 100,#100,
                    verbose_eval=100) #100)
    models.append(gbm)

Training until validation scores don't improve for 300 rounds
[100]	training's l2: 1.7285	valid_1's l2: 1.79002
[200]	training's l2: 1.4249	valid_1's l2: 1.51282
[300]	training's l2: 1.25619	valid_1's l2: 1.38777
[400]	training's l2: 1.14175	valid_1's l2: 1.32339
[500]	training's l2: 1.05464	valid_1's l2: 1.26608
[600]	training's l2: 0.985798	valid_1's l2: 1.22718
[700]	training's l2: 0.928841	valid_1's l2: 1.19618
[800]	training's l2: 0.879244	valid_1's l2: 1.16181
[900]	training's l2: 0.841594	valid_1's l2: 1.13917
[1000]	training's l2: 0.810244	valid_1's l2: 1.11513
[1100]	training's l2: 0.777291	valid_1's l2: 1.09244
[1200]	training's l2: 0.753447	valid_1's l2: 1.07734
[1300]	training's l2: 0.731369	valid_1's l2: 1.06069
[1400]	training's l2: 0.714583	valid_1's l2: 1.05059
[1500]	training's l2: 0.696367	valid_1's l2: 1.0338
[1600]	training's l2: 0.678759	valid_1's l2: 1.02048
[1700]	training's l2: 0.660927	valid_1's l2: 1.00647
[1800]	training's l2: 0.646317	valid_1's l2: 0.994278


[18400]	training's l2: 0.263119	valid_1's l2: 0.759441
[18500]	training's l2: 0.262613	valid_1's l2: 0.759273
[18600]	training's l2: 0.262043	valid_1's l2: 0.758876
[18700]	training's l2: 0.261573	valid_1's l2: 0.758746
[18800]	training's l2: 0.261066	valid_1's l2: 0.758505
[18900]	training's l2: 0.260536	valid_1's l2: 0.758343
[19000]	training's l2: 0.260093	valid_1's l2: 0.758146
[19100]	training's l2: 0.259612	valid_1's l2: 0.758174
[19200]	training's l2: 0.25908	valid_1's l2: 0.757965
[19300]	training's l2: 0.258542	valid_1's l2: 0.757776
[19400]	training's l2: 0.258011	valid_1's l2: 0.757583
[19500]	training's l2: 0.257541	valid_1's l2: 0.757579
[19600]	training's l2: 0.257004	valid_1's l2: 0.757432
[19700]	training's l2: 0.256481	valid_1's l2: 0.757207
[19800]	training's l2: 0.255874	valid_1's l2: 0.756676
[19900]	training's l2: 0.255457	valid_1's l2: 0.756408
[20000]	training's l2: 0.254971	valid_1's l2: 0.756373
[20100]	training's l2: 0.254587	valid_1's l2: 0.756155
[20200]	tra

In [ ]:
%%time
import matplotlib.pyplot as plt
import seaborn as sns

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(), gbm.feature_name()),reverse = True), columns=['Value','Feature'])
plt.figure(figsize=(10, 5))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM FEATURES')
plt.tight_layout()
plt.show()

In [ ]:
%%time
i = 0
res = []
step_size = 50000
for j in tqdm(range(int(np.ceil(test_df.shape[0] / 50000)))):
    res.append(np.expm1(sum([model.predict(test_df.iloc[i:i + step_size]) for model in models]) / folds))
    i += step_size

In [ ]:
%%time
from datetime import datetime

res = np.concatenate(res)
sample_submission["meter_reading"] = res
sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0
sample_submission.to_csv('sub_' + str(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')) + '.csv', index=False)
sample_submission.head(10)

In [ ]:
1